In [27]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from scipy.spatial.distance import cosine

# Define image transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize to the input size required by ResNet
    transforms.ToTensor(),           # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize as required by ResNet
])

# Load the pretrained ResNet50 model
model = models.resnet50(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])  # Remove the final classification layer
model.eval()  # Set the model to evaluation mode

# Function to extract features from an image
def extract_features(image_path, model):
    image = Image.open(image_path).convert('RGB')  # Load the image
    image = transform(image).unsqueeze(0)  # Apply transformations and add a batch dimension
    with torch.no_grad():
        features = model(image)  # Extract features
    return features.squeeze().numpy()  # Return feature vector as a numpy array

# Function to compare two images
def compare_images(img1_path, img2_path, model):
    # Extract features for both images
    features_img1 = extract_features(img1_path, model)
    features_img2 = extract_features(img2_path, model)

    # Calculate cosine similarity between the two feature vectors
    similarity = 1 - cosine(features_img1, features_img2)  # 1 - cosine gives similarity
    return similarity

# Paths to two images
image1 = "face_1.jpeg"
image2 = "9.jpeg"

# Compare the images
similarity = compare_images(image1, image2, model)
print(f"Cosine Similarity: {similarity:.4f}")

if similarity > 0.5:  # Threshold for similarity
    print("The images are similar.")
else:
    print("The images are not similar.")


Cosine Similarity: 0.5417
The images are similar.


In [9]:

import os
from glob import glob
from PIL import Image
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import random

c:\Users\keshav\anaconda3\envs\cuda_test\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
import os
import random
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt

# Define the folder containing images
image_folder = 'image_dataset'

# Get a list of all image files in the folder
image_files = glob(os.path.join(image_folder, '*.jpg'))

# Ensure the sample size does not exceed the number of available images
sample_size = min(10, len(image_files))

# Randomly select images
random.seed(42)
selected_images = random.sample(image_files, sample_size)

# Display the selected images
plt.figure(figsize=(20, 10))
for i, image_path in enumerate(selected_images):
    img = Image.open(image_path)
    plt.subplot(2, 5, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()


<Figure size 2000x1000 with 0 Axes>

In [11]:
def generate_clip_embeddings(images_path, model):

    image_paths = glob(os.path.join(images_path, '**/*.jpg'), recursive=True)

    embeddings = []
    for img_path in image_paths:
        image = Image.open(img_path)
        embedding = model.encode(image)
        embeddings.append(embedding)

    return embeddings, image_paths



IMAGES_PATH = 'E:/STGI/Dataset/test/FAKE'
model = SentenceTransformer('clip-ViT-B-32')
# embeddings, image_paths = generate_clip_embeddings(IMAGES_PATH, model)

In [15]:
import faiss
import numpy as np

def create_faiss_index_ivfpq(embeddings, image_paths, output_path, 
                             nlist=1000, m=16, nbits=8, 
                             training_size=100000):
    """
    Creates a FAISS IndexIVFPQ index for efficient similarity search.

    Parameters:
    - embeddings (List[List[float]]): List of image feature vectors.
    - image_paths (List[str]): Corresponding image file paths.
    - output_path (str): Path to save the FAISS index.
    - nlist (int): Number of clusters for IVF.
    - m (int): Number of PQ subquantizers.
    - nbits (int): Number of bits per subquantizer.
    - training_size (int): Number of vectors to use for training.

    Returns:
    - index (faiss.Index): Trained FAISS index.
    """
    # Convert embeddings to NumPy array
    vectors = np.array(embeddings).astype('float32')
    dimension = vectors.shape[1]

    # Define the quantizer (IndexFlatIP for inner product)
    quantizer = faiss.IndexFlatIP(dimension)

    # Initialize the IndexIVFPQ
    index_ivfpq = faiss.IndexIVFPQ(quantizer, dimension, nlist, m, nbits)

    # Wrap with IndexIDMap to handle custom IDs
    index = faiss.IndexIDMap(index_ivfpq)

    # Training the index
    if not index_ivfpq.is_trained:
        # Select a random subset for training
        np.random.seed(123)  # For reproducibility
        if training_size > len(vectors):
            training_size = len(vectors)
        random_indices = np.random.choice(len(vectors), size=training_size, replace=False)
        training_vectors = vectors[random_indices]
        print(f"Training the IndexIVFPQ with {training_size} vectors...")
        index_ivfpq.train(training_vectors)
        print("Training completed.")

    # Assign unique IDs to each vector
    ids = np.arange(len(embeddings)).astype('int64')

    # Add vectors with their IDs to the index
    print("Adding vectors to the index...")
    index.add_with_ids(vectors, ids)
    print(f"Total vectors indexed: {index.ntotal}")

    # Save the index
    faiss.write_index(index, output_path)
    print(f"Index created and saved to {output_path}")

    # Save image paths with their corresponding IDs
    with open(output_path + '.paths', 'w') as f:
        for img_path in image_paths:
            f.write(img_path + '\n')

    return index



OUTPUT_INDEX_PATH = "vector.index"
index = create_faiss_index_ivfpq(embeddings, image_paths, OUTPUT_INDEX_PATH)

Training the IndexIVFPQ with 10000 vectors...
Training completed.
Adding vectors to the index...
Total vectors indexed: 10000
Index created and saved to vector.index


In [4]:
OUTPUT_INDEX_PATH = "vector.index"
import faiss

def load_faiss_index(index_path):
    index = faiss.read_index(index_path)
    with open(index_path + '.paths', 'r') as f:
        image_paths = [line.strip() for line in f]
    print(f"Index loaded from {index_path}")
    return index, image_paths

index, image_paths = load_faiss_index(OUTPUT_INDEX_PATH)

Index loaded from vector.index


In [5]:
def retrieve_similar_images(query, model, index, image_paths, top_k=3):

    if query.endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        query = Image.open(query)

    query_features = model.encode(query)
    query_features = query_features.astype(np.float32).reshape(1, -1)

    distances, indices = index.search(query_features, top_k)

    retrieved_images = [image_paths[int(idx)] for idx in indices[0]]

    return query, retrieved_images

In [6]:
def visualize_results(query, retrieved_images):
    plt.figure(figsize=(12, 5))

    # If image query
    if isinstance(query, Image.Image):
        plt.subplot(1, len(retrieved_images) + 1, 1)
        plt.imshow(query)
        plt.title("Query Image")
        plt.axis('off')
        start_idx = 2

    # If text query
    else:
        plt.subplot(1, len(retrieved_images) + 1, 1)
        plt.text(0.5, 0.5, f"Query:\n\n '{query}'", fontsize=16, ha='center', va='center')
        plt.axis('off')
        start_idx = 2

    # Display images
    for i, img_path in enumerate(retrieved_images):

        plt.subplot(1, len(retrieved_images) + 1, i + start_idx)
        plt.imshow(Image.open(img_path))
        plt.title(f"Match {i + 1}")
        plt.axis('off')

    plt.show()


In [12]:

query = 'image_dataset\image_dataset\pexels-andrew-3201768.jpg'
query, retrieved_images = retrieve_similar_images(query, model, index, image_paths, top_k=3)
visualize_results(query, retrieved_images)

: 